In [1]:
import os 
import yaml
import logging

import numpy as onp 
import pandas as pd 
import time
import matplotlib.pyplot as plt
import seaborn as sns
import jax.random as random
import jax.numpy as np

from activitysim.core import mem
from activitysim.core import inject
from activitysim.core import tracing
from activitysim.core import config
from activitysim.core import pipeline
from activitysim.core import mp_tasks
from activitysim.core import chunk
from activitysim.core import orca

os.chdir('../../../../../PILATES')
from pilates.activitysim import preprocessor as asim_pre

os.chdir('../probaflow/')
from probaflow import dcm

os.chdir('../ual_asim/activitysim/austin/')

In [ ]:
logger = logging.getLogger('activitysim')
logger.setLevel(logging.DEBUG)

## Part 1
Run simulation until car ownership model 

In [34]:
def cleanup_output_files():

    active_log_files = \
        [h.baseFilename for h in logger.root.handlers if isinstance(
            h, logging.FileHandler)]
    tracing.delete_output_files('log', ignore=active_log_files)

    tracing.delete_output_files('h5')
    tracing.delete_output_files('csv')
    tracing.delete_output_files('txt')
    tracing.delete_output_files('yaml')
    tracing.delete_output_files('prof')

In [35]:
def run(run_list, injectables=None):

    if run_list['multiprocess']:
        logger.info("run multiprocess simulation")
        mp_tasks.run_multiprocess(run_list, injectables)
    else:
        logger.info("run single process simulation")
        pipeline.run(
            models=run_list['models'], resume_after=run_list['resume_after'])
        pipeline.close_pipeline()
        chunk.log_write_hwm()

In [71]:
#Define injectables
inject.add_injectable('data_dir', 'data')
inject.add_injectable('configs_dir', ['configs', 'configs/configs'])
inject.add_injectable('output_dir', 'output')
# inject.get_injectable('configs_dir')

#Injectables list 
injectables = ['configs_dir', 'data_dir', 'output_dir']

#Configs files
run_list = mp_tasks.get_run_list()

# Injectables to dict 
injectables = list(set(injectables) | set(['data_dir', 'configs_dir', 'output_dir']))
injectables = {k: inject.get_injectable(k) for k in injectables}

In [72]:
run_list['models'] = run_list['models'][:5] 
run_list['multiprocess_steps'][1]['models'] = run_list['multiprocess_steps'][1]['models'][:2]
run_list['multiprocess_steps'][1]['chunk_size'] =  500000000
run_list['multiprocess_steps'] = run_list['multiprocess_steps'][:2]

In [73]:
cleanup_output_files()
run(run_list, injectables)

############ mp_tasks - mp_initialize - mp_run_simulation mp_initialize
mp_initialize WARNING - activitysim.core.assign - numpy: Warning: underflow encountered in exp expression: _decay = _rt_available * exp(_nmDist * dispersion_parameter_walk)
############ mp_tasks - mp_households_0 - mp_run_simulation mp_households
############ mp_tasks - mp_households_1 - mp_run_simulation mp_households
############ mp_tasks - mp_households_2 - mp_run_simulation mp_households
############ mp_tasks - mp_households_3 - mp_run_simulation mp_households
############ mp_tasks - mp_households_4 - mp_run_simulation mp_households
############ mp_tasks - mp_households_5 - mp_run_simulation mp_households
############ mp_tasks - mp_households_6 - mp_run_simulation mp_households
############ mp_tasks - mp_households_7 - mp_run_simulation mp_households
############ mp_tasks - mp_households_8 - mp_run_simulation mp_households
############ mp_tasks - mp_households_9 - mp_run_simulation mp_households
############ mp

## Part 2 Runing calibration

In [ ]:
def add_simulation_vars(households, accessibilities, land_use, expressions):
    '''expressions should be a pandas series. Index is the name of the column'''

    df = hh.merge(acc, how = 'inner', left_on = 'TAZ', right_index = True).\
                  merge(lu, how = 'inner', left_on = 'TAZ', right_index = True)
    
    df['cars'] = df.VEHICL.where(df.VEHICL < 4, 4)
    
    expressions = dict(expressions)
    
    for label, expression in expressions.items():
#         print (expression)
        try:
            if expression.startswith('@'):
                df[label] = eval(expression[1:])
            else:
                df[label] = df.eval(expression)
        except NameError:
            df[label] = 0
    
    return df

In [ ]:
def car_ownership_specs(path_specs, path_coeffs):
    """ Reads the specification for the auto ownership model"""
    # Read csv file 
    specs_csv = pd.read_csv(path_specs).set_index('Label')
    coeffs_csv = pd.read_csv(path_coeffs).set_index('coefficient_name')
    return specs_csv, coeffs_csv

In [ ]:
def create_yml(specification, coeffs):
    
    # Select the alternative columns. Based alternative is dropped
    specs = specification.copy()
    vars_ = specs.columns[~specs.columns.isin(['Description', 'Expression'])]
    final_vars = []
    for var in vars_:
        if specs[var].isnull().all():
            pass
        else:
            final_vars.append(var)
            
    for var in final_vars:
        specs[var] = specs[var].replace(coeffs)
        
    # Adding constrains 
    constrains = specs[final_vars].copy()
    constrains[final_vars] = 2
    
    constrains[constrains.index.str.contains('util_asc')] = 1
    constrains[constrains.index.str.contains('util_driver')] = 1
    constrains[constrains.index.str.contains('util_asc_')] = 0
    constrains = constrains.astype(int)
    
    # Create specification for yml 
    final_dict = {}
    final_dict['model_coeffs'] = specs[final_vars].to_dict(orient = 'list')
    final_dict['spec_names'] = list(specs.index)
    final_dict['constrains'] = constrains.to_dict(orient = 'list')
    
    write_yaml(final_dict, 'car_ownership.yml')

In [ ]:
out_dir = 'output/'
hdf = pd.HDFStore(out_dir + 'pipeline.h5')
hh = pd.read_csv('output/final_households.csv')
acc = hdf['/accessibility/compute_accessibility']
lu = pd.read_csv('output/final_land_use.csv')
print ('Number of Households: ',hh.shape[0])

In [ ]:
#Specification
config_dir = 'configs/'
path_specs = config_dir +'configs/auto_ownership.csv'
path_coeffs = config_dir + 'configs/auto_ownership_coeffs.csv'

In [ ]:
coeffs_csv = pd.read_csv(path_coeffs).set_index('coefficient_name')

In [ ]:
a = pd.read_csv(path_specs).set_index('Label')

In [ ]:
specs, coeffs_csv = car_ownership_specs(path_specs, path_coeffs)
coeffs = coeffs_csv['value'].to_dict()
create_yml(specs, coeffs)